In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential, Model # initialize neural network library
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate

In [ ]:


# Any results you write to the current directory are saved as output.


from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing import text, sequence
from sklearn.metrics import f1_score
import os
print(os.listdir("../input"))

In [ ]:


train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("Train datasets shape:", train_df.shape)
print("Test datasets shape:", test_df.shape)

In [ ]:
## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen,value=0.)
val_X = pad_sequences(val_X, maxlen=maxlen,value=0.)
test_X = pad_sequences(test_X, maxlen=maxlen,value=0.)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values


In [ ]:
print(train_y.shape)
print(train_X.shape)
print(test_X)

In [ ]:
1306122*0.9

In [ ]:
print(val_X)

In [ ]:
def get_model2():    
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    print(model.summary())
    return model
model = get_model2()

In [ ]:
#Create model, train and predict
#model = Sequential()
model.fit(train_X,train_y)
pred_noemb_val_y = model.predict(val_X)
pred_noemb_test_y = model.predict(test_X)
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


pred_noemb_test_y.shape
train_meta = np.zeros(train_y.shape)
test_meta = np.zeros(test_X.shape[0])
y_pred_val = (pred_noemb_val_y > 0.5).astype(int)
y_pred_test = (pred_noemb_test_y > 0.5).astype(int)
# Print predict using f1score from sklearn
#score = f1_score(val_y,y_val)
#print(score)

In [ ]:
score = f1_score(val_y,y_pred_val)
print(score)

In [ ]:
# Collect garbage
import time
import gc; gc.collect()
time.sleep(10)

In [ ]:
# Create a submission
sub_df = pd.DataFrame({'qid':test_df.qid.values})
sub_df['prediction'] = y_pred_test
sub_df.to_csv('submission.csv', index=False)